In [8]:
# 1. Java 설치 (한 번만 실행)
!apt-get update
!apt-get install -y openjdk-8-jdk

# 2. pip 최신화 & JPype1 설치
!pip install --upgrade pip
!pip install --upgrade setuptools wheel
!pip install JPype1==1.4.1

# 3. KoNLPy 설치 (최신버전)
!pip install konlpy

# 4. (선택) WordCloud 한글 폰트 설정
!apt-get install -y fonts-nanum


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,918 kB]
Fetched 2,307 kB in 14s (161 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 87.9 MB/s  0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (12.4 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 127184 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


# 군집 분류


군집 0: 음식, 기타 (김치, 불고기 등)


군집 1: 한국 관련, 기업, 도시


군집 2: 스포츠, 이벤트 등


In [2]:


from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch
from sklearn.cluster import KMeans

# 위키피디아 예시 문장들 (복수 주제)
sentences = [
    "대한민국의 수도는 서울이다.",
    "축구는 전 세계에서 인기 있는 스포츠다.",
    "삼성전자는 반도체와 스마트폰으로 유명하다.",
    "김치는 한국의 전통 음식이다.",
    "인공지능은 미래 산업의 핵심이다.",
    "올림픽은 세계인의 축제다.",
    "LG전자는 가전제품으로 잘 알려져 있다.",
    "농구는 미국에서 인기가 많다.",
    "불고기는 한국을 대표하는 요리다.",
    "딥러닝 기술은 다양한 분야에 적용되고 있다."
]

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")

def get_emb(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=64)
    with torch.no_grad():
        out = model(**tokens)
    return out.last_hidden_state[:,0,:].squeeze().numpy()

embeddings = [get_emb(s) for s in sentences]
print(embeddings)
# 군집화
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(embeddings)

# 결과 보기
df = pd.DataFrame({'문장': sentences, '군집': labels})
print(df)


[array([ 2.40075141e-02,  8.41708302e-01,  3.65448207e-01, -4.44704965e-02,
       -1.93864182e-01, -1.12606525e+00, -1.80536896e-01, -6.56688586e-02,
        2.77658701e-01, -4.61014092e-01,  4.31598723e-03, -1.01682186e+00,
       -4.06762481e-01, -4.18718532e-02,  3.48695725e-01, -2.41316214e-01,
       -6.81035876e-01,  5.04779279e-01,  5.55829465e-01, -1.29199773e-01,
       -8.56167972e-02, -1.25496276e-02, -1.84290081e-01, -5.12910247e-01,
       -9.79974419e-02,  9.17986855e-02, -2.78800368e-01,  7.34593421e-02,
       -5.09453118e-01,  1.28240392e-01, -5.62998839e-02, -3.29214990e-01,
        4.24462080e+00,  9.61841583e-01, -3.38529378e-01,  6.38110101e-01,
        2.03903437e-01,  3.95014405e-01, -1.54656217e-01, -2.41125584e-01,
       -3.97870988e-01,  1.71169281e-01, -1.33630857e-01,  5.18580005e-02,
        3.63182873e-02,  1.17245287e-01,  9.01016533e-01, -6.66250646e-01,
       -4.09240633e-01,  6.81014597e-01,  5.26345074e-01,  1.12163079e+00,
       -4.63291556e-02, 